# Tema 3: Matriz coocurrencias

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
import numpy as np
import pandas as pd

# Descargar recursos de NLTK
#nltk.download("punkt")
#nltk.download("stopwords")

## Ejercicio 1

In [2]:
# Texto de ejemplo
text = "Mysterious tunnels sketched by Leonardo da Vinci in the late 1400s may have been found at the Castle. Secret tunnels at the Sforza Castle."

In [3]:
# Se preprocesa el texto, eliminando stopwords, convirtiendo las palabras a minúsculas, tokenizando y eliminando los tokens no alfanuméricos
stop_words = set(stopwords.words("english"))
words = word_tokenize(text.lower())
words = [word for word in words if word.isalnum() and word not in stop_words]

In [4]:
# Se define el tamaño de ventana y se crea la lista de pares que coocurren dentro de la ventana

# Definir el tamaño de ventana para la coocurrencia
window_size = 2

# Crear una lista de pares de palabras que coocurren
co_occurrences = defaultdict(Counter)
for i, word in enumerate(words):
    for j in range(max(0, i - window_size), min(len(words), i + window_size + 1)):
        if i != j:
            co_occurrences[word][words[j]] += 1
            # print("palabra:  ",word, "  words[j]:  ", words[j])

In [5]:
# Se obtiene el vocabulario
# Crear una lista de palabras únicas
unique_words = sorted(set(words))

In [6]:
# Se crea la matriz de coocurrencia
# Inicializar la matriz de coocurrencias
co_matrix = np.zeros((len(unique_words), len(unique_words)), dtype=int)

# Poblar la matriz de coocurrencias
word_index = {word: idx for idx, word in enumerate(unique_words)}
for word, neighbors in co_occurrences.items():
    for neighbor, count in neighbors.items():
        co_matrix[word_index[word]][word_index[neighbor]] = count

In [7]:
# Se crea un DataFrame para una mejor visualización
# Crear un DataFrame para una mejor legibilidad
co_matrix_df = pd.DataFrame(co_matrix, index=unique_words, columns=unique_words)

# Mostrar la matriz de coocurrencias
co_matrix_df

,1400s,castle,da,found,late,leonardo,may,mysterious,secret,sforza,sketched,tunnels,vinci
1400s,0,0,0,1,1,0,1,0,0,0,0,0,1
castle,0,0,0,1,0,0,1,0,1,1,0,2,0
da,0,0,0,0,1,1,0,0,0,0,1,0,1
found,1,1,0,0,0,0,1,0,1,0,0,0,0
late,1,0,1,0,0,0,1,0,0,0,0,0,1
leonardo,0,0,1,0,0,0,0,0,0,0,1,1,1
may,1,1,0,1,1,0,0,0,0,0,0,0,0
mysterious,0,0,0,0,0,0,0,0,0,0,1,1,0
secret,0,1,0,1,0,0,0,0,0,1,0,1,0
sforza,0,1,0,0,0,0,0,0,1,0,0,1,0


## Ejercicio 2
Se quiere calcular la similitud coseno entre pares de palabras a partir de la matriz de coocurrencias anterior.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_sklearn(matrix, word1_index, word2_index):
    # Obtener los vectores de las dos palabras
    vector1 = matrix[word1_index].reshape(1, -1)
    vector2 = matrix[word2_index].reshape(1, -1)

    # Calcular la similitud del coseno usando sklearn
    similarity = cosine_similarity(vector1, vector2)[0][0]

    return similarity


# Calcular la similitud del coseno entre las palabras "da" y "vinci"
similarity = cosine_similarity_sklearn(co_matrix, word_index["da"], word_index["vinci"])

print(f"La similitud del coseno entre las dos palabras es: {similarity}")

# Calcular la similitud del coseno entre las palabras "may" y "tunnels"
similarity = cosine_similarity_sklearn(co_matrix, word_index["may"], word_index["tunnels"])

print(f"La similitud del coseno entre las dos palabras es: {similarity}")

# Calcular la similitud del coseno entre una palabra consigo misma
similarity = cosine_similarity_sklearn(co_matrix, word_index["tunnels"], word_index["tunnels"])

print(f"La similitud del coseno entre las dos palabras es: {similarity}")

# Calcular la similitud del coseno entre las palabras "tunnels" y "castle"
similarity = cosine_similarity_sklearn(co_matrix, word_index["tunnels"], word_index["castle"])

print(f"La similitud del coseno entre las dos palabras es: {similarity}")

# Calcular la similitud del coseno entre las palabras "secret" y "tunnels"
similarity = cosine_similarity_sklearn(co_matrix, word_index["secret"], word_index["tunnels"])

print(f"La similitud del coseno entre las dos palabras es: {similarity}")


La similitud del coseno entre las dos palabras es: 0.5
La similitud del coseno entre las dos palabras es: 0.3333333333333333
La similitud del coseno entre las dos palabras es: 1.0000000000000002
La similitud del coseno entre las dos palabras es: 0.2357022603955158
La similitud del coseno entre las dos palabras es: 0.5


## Ejercicio 3
Se quiere calcular las palabras más similares a una dada a partir de la matriz de coocurrencias anterior.

In [9]:
def most_similar_words(matrix, word_index, n):
    # Obtener el vector de la palabra dada
    word_vector = matrix[word_index].reshape(1, -1)

    # Calcular la similitud del coseno entre la palabra dada y todas las demás palabras
    similarities = cosine_similarity(matrix, word_vector).flatten()

    # Obtener los índices de las n palabras más similares (excluyendo la palabra dada)
    most_similar_indices = similarities.argsort()[-n - 1 : -1][::-1]

    return most_similar_indices


n_words_most_similar = 5

# Obtener los índices de las 3 palabras más similares a la palabra en el índice que corresponda

word = "leonardo"
similar_words_indices = most_similar_words(co_matrix, word_index[word], n_words_most_similar)

print("Las palabras más similares a ", word, " son: {similar_words_indices}")

for index in similar_words_indices:
    print(unique_words[index])


word = "da"
similar_words_indices = most_similar_words(co_matrix, word_index[word], n_words_most_similar)

print("Las palabras más similares a ", word, " son: {similar_words_indices}")

for index in similar_words_indices:
    print(unique_words[index])

Las palabras más similares a  leonardo  son: {similar_words_indices}
mysterious
late
da
sketched
castle
Las palabras más similares a  da  son: {similar_words_indices}
leonardo
1400s
vinci
mysterious
tunnels


## Ejercicio 4

Se quiere calcular las palabras más similares a una dada utilizando como pesado la función PPMI.

In [10]:
def create_ppmi_matrix(cooccurrence_matrix):
    # Calcular la suma total de la matriz de coocurrencias
    total_sum = np.sum(cooccurrence_matrix)

    # Calcular la suma de cada fila y cada columna
    row_sums = np.sum(cooccurrence_matrix, axis=1)
    col_sums = np.sum(cooccurrence_matrix, axis=0)

    # Inicializar la matriz PPMI con ceros
    ppmi_matrix = np.zeros_like(cooccurrence_matrix, dtype=float)

    # Calcular los valores PPMI para cada celda en la matriz
    for i in range(cooccurrence_matrix.shape[0]):
        for j in range(cooccurrence_matrix.shape[1]):
            p_ij = cooccurrence_matrix[i, j] / total_sum
            p_i = row_sums[i] / total_sum
            p_j = col_sums[j] / total_sum

            if p_ij > 0:
                ppmi_value = max(0, np.log2(p_ij / (p_i * p_j)))
                ppmi_matrix[i, j] = ppmi_value

    return ppmi_matrix


# Crear la matriz PPMI
ppmi_matrix = create_ppmi_matrix(co_matrix)

print("Matriz de coocurrencias:")
print(co_matrix)

print("\nMatriz PPMI:")
print(ppmi_matrix)

n_words_most_similar = 5

# Obtener los índices de las 3 palabras más similares a la palabra en el índice que corresponda

word = "leonardo"
similar_words_indices = most_similar_words(ppmi_matrix, word_index[word], n_words_most_similar)

print("Las palabras más similares a ", word, " son: {similar_words_indices}")

for index in similar_words_indices:
    print(unique_words[index])


word = "da"
similar_words_indices = most_similar_words(ppmi_matrix, word_index[word], n_words_most_similar)

print("Las palabras más similares a ", word, " son: {similar_words_indices}")

for index in similar_words_indices:
    print(unique_words[index])


Matriz de coocurrencias:
[[0 0 0 1 1 0 1 0 0 0 0 0 1]
 [0 0 0 1 0 0 1 0 1 1 0 2 0]
 [0 0 0 0 1 1 0 0 0 0 1 0 1]
 [1 1 0 0 0 0 1 0 1 0 0 0 0]
 [1 0 1 0 0 0 1 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 1 1 1]
 [1 1 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 1 0 1 0 0 0 0 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 1 0 0 1 0]
 [0 0 1 0 0 1 0 1 0 0 0 1 0]
 [0 2 0 0 0 1 0 1 1 1 1 0 0]
 [1 0 1 0 1 1 0 0 0 0 0 0 0]]

Matriz PPMI:
[[0.         0.         0.         1.7548875  1.7548875  0.
  1.7548875  0.         0.         0.         0.         0.
  1.7548875 ]
 [0.         0.         0.         1.169925   0.         0.
  1.169925   0.         1.169925   1.5849625  0.         1.36257008
  0.        ]
 [0.         0.         0.         0.         1.7548875  1.7548875
  0.         0.         0.         0.         1.7548875  0.
  1.7548875 ]
 [1.7548875  1.169925   0.         0.         0.         0.
  1.7548875  0.         1.7548875  0.         0.         0.
  0.        ]
 [1.7548875  0.         1.754887